In [1]:
import time
from datetime import datetime

import numpy as np
import pandas as pd
import requests
import json

In [2]:
START_2021 = 1609459200

In [3]:
def datetime_to_unix(dt):
    d = datetime.strptime(dt, '%Y-%m-%dT%H:%M:%S.%fZ')
    return int(time.mktime(d.timetuple()))

In [12]:
API_TOKEN = 'BQCEGpPK4qhilj657AI_9BK25G6pvPuMiLyTACmEUTXM-1RLqhUVMuAI9SR9nNIcelDM3yDwrYduEpjt03-cjCJ5s_daz-G3T4H5i9tProkh66fuZkl1Oc0-jCVqnjJFcAW0vUiY0f6cSA0dNyIYjYsxRXjnPupBZxap5bKyoA'

In [33]:
def get_recent_songs(limit=50, before=None, token=API_TOKEN, url=None):
    headers = {
        'Authorization': 'Bearer {token}'.format(token=token)
    }
        
    if url == None:
        url = 'https://api.spotify.com/v1/me/player/recently-played'
        data = {
            'limit' : limit
        }
        if before != None:
            data['before'] = before

        response = requests.get(url, headers=headers, params=data)
    else:
        response = requests.get(url, headers=headers)
    
    response = json.loads(response.content)
    return response

In [14]:
response = get_recent_songs(limit=50)

In [16]:
response['cursors']

{'after': '1625221076128', 'before': '1625069915350'}

In [17]:
def response_items_to_df(response_items):
    ids = [x['track']['id'] for x in response_items]
    played_ats = [x['played_at'] for x in response_items]
    artists = [x['track']['artists'] for x in response_items]
    duration_mss = [x['track']['duration_ms'] for x in response_items]
    names = [x['track']['name'] for x in response_items]

    df = pd.DataFrame(np.array([ids, played_ats, artists, duration_mss, names], dtype=object).T, columns=['id', 'played_at', 'artists', 'duration_ms', 'name'])

    df['played_at'] = df.played_at.apply(lambda x: datetime_to_unix(x))
    
    return df

In [34]:
res = get_recent_songs(limit=50, url='https://api.spotify.com/v1/me/player/recently-played?before=1625069915350&limit=50')
df = response_items_to_df(res['items'])
print(response['cursors'])
print(response['next'])
display(df)

{'after': '1625221076128', 'before': '1625069915350'}
https://api.spotify.com/v1/me/player/recently-played?before=1625069915350&limit=50


,id,played_at,artists,duration_ms,name


In [119]:
dfs = []
earliest_time = None
before_cursor = None

while (earliest_time == None) or (earliest_time > START_2021):
    response = get_recent_songs(limit=50, before=before_cursor)
    latest_df = response_items_to_df(response['items'])
    dfs.append(latest_df)
    earliest_time = latest_df.played_at.min()
    
    if response['cursors'] != None:
        before_cursor = int(response['cursors']['after']) - 1
    else:
        break

df = pd.concat(dfs)

In [120]:
df

,id,played_at,artists,duration_ms,name
0,1aU1wpYBSpP0M6IiihY5Ue,1625217220,[{'external_urls': {'spotify': 'https://open.s...,254466,Renegade (feat. Taylor Swift)
1,1aU1wpYBSpP0M6IiihY5Ue,1625216965,[{'external_urls': {'spotify': 'https://open.s...,254466,Renegade (feat. Taylor Swift)
2,1aU1wpYBSpP0M6IiihY5Ue,1625216638,[{'external_urls': {'spotify': 'https://open.s...,254466,Renegade (feat. Taylor Swift)
3,1aU1wpYBSpP0M6IiihY5Ue,1625216383,[{'external_urls': {'spotify': 'https://open.s...,254466,Renegade (feat. Taylor Swift)
4,1aU1wpYBSpP0M6IiihY5Ue,1625216128,[{'external_urls': {'spotify': 'https://open.s...,254466,Renegade (feat. Taylor Swift)
...,...,...,...,...,...
1,0cqRj7pUJDkTCEsJkx8snD,1625066315,[{'external_urls': {'spotify': 'https://open.s...,219200,Shake It Off
2,55n9yjI6qqXh5F2mYvUc2y,1625066095,[{'external_urls': {'spotify': 'https://open.s...,247160,I Don’t Wanna Live Forever (Fifty Shades Darker)
0,0cqRj7pUJDkTCEsJkx8snD,1625066315,[{'external_urls': {'spotify': 'https://open.s...,219200,Shake It Off
1,55n9yjI6qqXh5F2mYvUc2y,1625066095,[{'external_urls': {'spotify': 'https://open.s...,247160,I Don’t Wanna Live Forever (Fifty Shades Darker)
